In [1]:
import tensorflow.compat.v2 as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pathlib
import random
import glob
import os
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import time
keras = tf.keras
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32)
    image = image/255.0
    image = 2*image-1
    return image

In [3]:
train_dir = r'C:\Users\24306\Desktop\datasets\kfold\3\train'
test_dir  = r'C:\Users\24306\Desktop\datasets\kfold\3\test'
checkpoint_path = r'C:\Users\24306\Desktop\NASNetMobile\3\NASNetMobile.ckpt'

# 处理训练数据

In [4]:
#打乱顺序
train_root = pathlib.Path(train_dir)
train_image_paths = list(train_root.glob('*/*'))
train_image_paths = [str(path) for path in train_image_paths]
random.shuffle(train_image_paths)
#处理训练标签
train_image_labels = [pathlib.Path(path).parent.name for path in train_image_paths]
train_labels = [[label.split('_')[0]] for label in train_image_labels]
new_train_labels=[]
for n in train_labels:
    new_train_labels.append(float(n[0]));  
train_labels=new_train_labels
print(train_labels)
#处理训练图像
train_path_ds = tf.data.Dataset.from_tensor_slices(train_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_image_ds = train_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
train_label_ds = tf.data.Dataset.from_tensor_slices(train_labels)
train_image_label_ds = tf.data.Dataset.zip((train_image_ds, train_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
train_count = len(train_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
train_data = train_image_label_ds.shuffle(buffer_size=train_count).repeat(-1)
train_data = train_data.batch(BATCH_SIZE)
train_data = train_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(train_data)



[84.0, 100.0, 37.0, 10.0, 93.0, 37.0, 95.0, 47.0, 86.0, 86.0, 66.0, 80.0, 80.0, 84.0, 45.0, 86.0, 4.0, 55.0, 52.0, 84.0, 66.0, 47.0, 37.0, 7.0, 35.0, 40.0, 37.0, 37.0, 66.0, 52.0, 100.0, 66.0, 10.0, 13.0, 55.0, 95.0, 16.0, 84.0, 80.0, 47.0, 45.0, 66.0, 55.0, 86.0, 84.0, 95.0, 37.0, 32.0, 86.0, 84.0, 45.0, 37.0, 45.0, 86.0, 13.0, 13.0, 86.0, 93.0, 100.0, 96.0, 16.0, 100.0, 66.0, 66.0, 93.0, 80.0, 47.0, 84.0, 45.0, 86.0, 4.0, 39.0, 100.0, 66.0, 80.0, 55.0, 45.0, 16.0, 45.0, 37.0, 37.0, 10.0, 37.0, 7.0, 52.0, 37.0, 35.0, 37.0, 80.0, 30.0, 93.0, 100.0, 80.0, 7.0, 39.0, 23.0, 13.0, 40.0, 95.0, 20.0, 10.0, 84.0, 10.0, 37.0, 45.0, 86.0, 40.0, 96.0, 13.0, 37.0, 16.0, 100.0, 100.0, 16.0, 4.0, 47.0, 37.0, 47.0, 30.0, 13.0, 47.0, 96.0, 95.0, 47.0, 66.0, 7.0, 37.0, 100.0, 37.0, 40.0, 37.0, 4.0, 16.0, 66.0, 80.0, 52.0, 84.0, 45.0, 86.0, 30.0, 100.0, 40.0, 16.0, 37.0, 93.0, 80.0, 96.0, 66.0, 7.0, 13.0, 95.0, 100.0, 100.0, 80.0, 100.0, 20.0, 100.0, 47.0, 16.0, 86.0, 30.0, 96.0, 30.0, 96.0, 96.0, 66.0

# 处理验证数据

In [5]:
#打乱顺序
test_root = pathlib.Path(test_dir)
test_image_paths = list(test_root.glob('*/*'))
test_image_paths = [str(path) for path in test_image_paths]
#random.shuffle(train_image_paths)
#处理训练标签
test_image_labels = [pathlib.Path(path).parent.name for path in test_image_paths]
test_labels = [[label.split('_')[0]] for label in test_image_labels]
new_test_labels=[]
for n in test_labels:
    new_test_labels.append(float(n[0]));  
test_labels=new_test_labels
print(test_labels)
#处理训练图像
test_path_ds = tf.data.Dataset.from_tensor_slices(test_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
test_image_ds = test_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
test_label_ds = tf.data.Dataset.from_tensor_slices(test_labels)
test_image_label_ds = tf.data.Dataset.zip((test_image_ds, test_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
test_count = len(test_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
test_data = test_image_label_ds.shuffle(buffer_size=test_count).repeat(-1)
test_data = test_data.batch(BATCH_SIZE)
test_data = test_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(test_data)

[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.

In [6]:
#要改
NASNetMobile = tf.keras.applications.NASNetMobile(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
#要改
NASNetMobile.trainable = True
inputs = tf.keras.Input(shape=(224, 224, 3))
#要改
x = NASNetMobile(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
out_weight = tf.keras.layers.Dense(1,name='out_weight')(x)

model = tf.keras.Model(inputs=inputs,
                       outputs=[out_weight])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 NASNet (Functional)         (None, 7, 7, 1056)        4269716   
                                                                 
 global_average_pooling2d (G  (None, 1056)             0         
 lobalAveragePooling2D)                                          
                                                                 
 out_weight (Dense)          (None, 1)                 1057      
                                                                 
Total params: 4,270,773
Trainable params: 4,234,035
Non-trainable params: 36,738
_________________________________________________________________


In [7]:
print ('start time:',time.asctime( time.localtime(time.time()) ))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, decay=0.01),loss='mse')

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,save_best_only=True,monitor='val_loss')
history = model.fit(train_data,
                    steps_per_epoch=train_count//BATCH_SIZE,
                    epochs=300,
                    validation_data=test_data,
                    validation_steps=test_count//BATCH_SIZE,
                   callbacks=[cp_callback])
#要改
np.savetxt('NASNetMobile_loss_3.txt',history.history['loss'])
#要改
np.savetxt('NASNetMobile_valloss_3.txt',history.history['val_loss'])
print ('end time:',time.asctime( time.localtime(time.time()) ))

start time: Sun Apr  9 09:21:27 2023
Epoch 1/300
591/591 [==============================] - ETA: 0s - loss: 66.3830

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetMobile\3\NASNetMobile.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetMobile\3\NASNetMobile.ckpt\assets


591/591 [==============================] - 191s 276ms/step - loss: 66.3830 - val_loss: 424.8884
Epoch 2/300
591/591 [==============================] - 85s 143ms/step - loss: 2.4356 - val_loss: 591.3683
Epoch 3/300
591/591 [==============================] - ETA: 0s - loss: 0.9818

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetMobile\3\NASNetMobile.ckpt\assets


...

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetMobile\3\NASNetMobile.ckpt\assets


591/591 [==============================] - 218s 369ms/step - loss: 0.0421 - val_loss: 0.2642
Epoch 102/300
591/591 [==============================] - 130s 220ms/step - loss: 0.0342 - val_loss: 0.3354
Epoch 103/300
591/591 [==============================] - 129s 218ms/step - loss: 0.0348 - val_loss: 0.4069
Epoch 104/300
591/591 [==============================] - 128s 217ms/step - loss: 0.0332 - val_loss: 0.2965
Epoch 105/300
591/591 [==============================] - 129s 217ms/step - loss: 0.0354 - val_loss: 0.3257
Epoch 106/300
591/591 [==============================] - 128s 217ms/step - loss: 0.0387 - val_loss: 0.3866
Epoch 107/300
591/591 [==============================] - 129s 219ms/step - loss: 0.0327 - val_loss: 0.3662
Epoch 108/300
591/591 [==============================] - 128s 217ms/step - loss: 0.0457 - val_loss: 0.3552
Epoch 109/300
591/591 [==============================] - 129s 218ms/step - loss: 0.0348 - val_loss: 0.3787
Epoch 110/300
591/591 [============================

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetMobile\3\NASNetMobile.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetMobile\3\NASNetMobile.ckpt\assets


591/591 [==============================] - 225s 381ms/step - loss: 0.0322 - val_loss: 0.2620
Epoch 230/300
591/591 [==============================] - 140s 237ms/step - loss: 0.0432 - val_loss: 0.2975
Epoch 231/300
591/591 [==============================] - 140s 237ms/step - loss: 0.0245 - val_loss: 0.2829
Epoch 232/300
591/591 [==============================] - 139s 236ms/step - loss: 0.0266 - val_loss: 0.2793
Epoch 233/300
591/591 [==============================] - 139s 235ms/step - loss: 0.0424 - val_loss: 0.3253
Epoch 234/300
591/591 [==============================] - 139s 235ms/step - loss: 0.0235 - val_loss: 0.3300
Epoch 235/300
591/591 [==============================] - 139s 236ms/step - loss: 0.0243 - val_loss: 0.3583
Epoch 236/300
591/591 [==============================] - 139s 235ms/step - loss: 0.0254 - val_loss: 0.3479
Epoch 237/300
591/591 [==============================] - 139s 235ms/step - loss: 0.0232 - val_loss: 0.3462
Epoch 238/300
591/591 [============================